# Configuraciones para la conexión con la base de datos

## Creación de la conexión con la instancia de PostgreSQL creada en Docker

Antes de iniciar con el desarrollo de las consultas, nos conectamos a la instancia de PostgreSQL creada dentro el contenedor en Docker.

Iniciamos verificando los procesos que se están ejecutando en el contenedor.

```PowerShell
docker ps 
```
<img src="assets/Conexion1.png" alt="Texto alternativo" width="600px" class="bg-primary" style="display: block; margin: 0 auto;">
<p style="text-align: center; width: 100%;">1. Procesos siendo ejecutados por nuestro contenedor **Docker**</p>

```{image} assets/Conexion1.png
:height: 150px
:width: 200px
:align: center
:name: procesos_en ejecucion_Docker

Procesos siendo ejecutados por nuestro contenedor **Docker**
```

Para **conectarnos a la instancia de PostgreSQL** creada por el contenedor, utilizamos el siguiente comando:


```{glue} PowerShell
docker exec -it dbtask1-postgres bash 
```

**docker exec** se encarga de ejecutar un comando de forma interactiva en un contenedor en ejecución, en este caso, el contenedor para PostgreSQL previamente instanciado, utilizando el programa **bash** dentro del contenedor. Luego de hacerlo, podemos conectarnos a **postgres** desde el **bash** utilizando el siguiente comando:



```PowerShell
:psql -U dbtask1 --password --db dbtask1
```

```{figure} Conexion2.png
height: 150px
:width: 200px
:align: center
name: conexion_db

Conexión a base de datos. La contraseña es suministrada en el prompt
```

## Administración de la base de datos con pgAdmin

Para visualizar las tablas creadas, se crea una conexión con pgAdmin, siguiendo los pasos explicados en clase.

```{figure} pgAdmin1.png
height: 150px
:width: 200px
:align: center
name: conexion_db

Página de inicio de pgAdmin
```

```{figure} pgAdmin2.png
height: 150px
:width: 200px
:align: center
name: conexion_db

Ingreso de credenciales para conectarnos a la base de datos instanciada.
```

```{figure} pgAdmin3.png
height: 150px
:width: 200px
:align: center
name: conexion_db

Base de datos del proyecto conectada.
```


## Configuración del entorno Python

Como lo indica el punto 4 de la asignación, las consultas SQL deben realizarse desde Python usando la API psycopg2. Para ello, inicialmente crearemos un entorno aislado con las dependencias que serán utilizadas en el marco del proyecto.

```PowerShell
conda create --name tarea1_venv python=3.9
```

Ahora para activarlos, usamos:

```PowerShell
conda activate tarea1_venv
```

Instalamos librerías básicas para el análisis de datos y el desarrollo de las consultas.

```PowerShell
pip install -U jupyter-book
pip install -U psycopg2
pip install -U numpy
pip install -U scipy
pip install -U pandas
pip install -U matplotlib
pip install -U scikit-learn
pip install -U jupyter-book
pip install -U psycopg2
pip install jupyter lab
```

Ahora, instalamos un nuevo kernel de IPython con el nombre "tarea1_venv" en el entorno de mi usuario actual. Esto con el propósito de utilizar este kernel específico en mis Jupyter notebooks para trabajar en el entorno virtual creado para el proyecto.

```PowerShell
pip install -U ipykernel
python -m ipykernel install --user --name=tarea1_venv
```

En este caso, las consultas las desarrollamos en Jupyter lab. 

```PowerShell
jupyter lab
```



```sh
git clone https://github.com/TomasBeuzen/python-programming-for-data-science.git
```

# Python API para PosgreSQL

La biblioteca psycopg2 es un controlador de base de datos para PostgreSQL en Python. Para el propósito de la actividad, importamos la librería y la clase Error para manejar errores específicos de PostgreSQL

In [1]:

import psycopg2
from psycopg2 import Error

Ahora intentamos establecer una conexión con la base de datos PostgreSQL creada. Utilizamo los parámetros de conexión, como el nombre de usuario, contraseña, host, puerto y nombre de la base de datos, para conectarnos a un servidor local. Si la conexión se establece con éxito, creamos un objeto cursor que se utilizará para ejecutar comandos SQL en la base de datos.

In [8]:
try:
    connection = psycopg2.connect(user="dbtask1",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="dbtask1")

    cursor = connection.cursor()
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")

    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
#finally:
    #if (connection):
        #cursor.close()
        #connection.close()
        #print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'dbtask1', 'channel_binding': 'prefer', 'dbname': 'dbtask1', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 



## Consultas SQL

### I. CREATE TABLE employees

Este código crea una nueva tabla llamada "employees" en una base de datos y establece un conjunto de reglas y restricciones en esta tabla para garantizar la integridad de los datos, como la unicidad del ID de empleado.

In [5]:

cursor.execute('''DROP TABLE IF EXISTS employees''')
cursor.execute('''CREATE TABLE employees
   ( employee_id INTEGER
   , first_name VARCHAR(20)
   , last_name VARCHAR(25)
   , email VARCHAR(25)
   , phone_number VARCHAR(20)
   , hire_date DATE
   , job_id VARCHAR(10)
   , salary NUMERIC(8,2)
   , commission_pct NUMERIC(2,2)
   , manager_id INTEGER
   , department_id INTEGER
   ) ;

CREATE UNIQUE INDEX emp_emp_id_pk
         ON employees (employee_id) ;

ALTER TABLE employees ADD
   PRIMARY KEY (employee_id);
''')

# Confirma los cambios en la base de datos
connection.commit()

Usando la sentencia a continuación, seleccionamos todos los registros de la tabla "employees". Luego, utiliza la cláusula "LIMIT 10" para restringir el resultado a solo 10 filas.

In [6]:
# Ejecutar la consulta
cursor.execute("SELECT * FROM employees LIMIT 10")

# Recuperar los resultados
rows = cursor.fetchall()

# Iterar sobre los resultados
for row in rows:
    print(row)
print("Operation done successfully")

Operation done successfully


Ahora, empleando esta sentencia hacemos una agregación para contar el número total de registros en la tabla "employees". La consulta devolverá un solo valor que representa la cantidad total de registros en la tabla "employees".

In [7]:
# Ejecutar la consulta
cursor.execute("SELECT count(1) FROM employees")

# Recuperar el resultado
count = cursor.fetchone()[0]

# Imprimir el resultado
print(f"El número de empleados es: {count}")

El número de empleados es: 0


### II. CREATE TABLE courses